# Project 4: Analysing variants with BigQuery

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


## Provide your credentials to the runtime and set project environment

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%env GCLOUD_PROJECT=boxwood-bee-258411

env: GCLOUD_PROJECT=boxwood-bee-258411


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [0]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

In [0]:
%unload_ext google.colab.data_table

In [0]:
import pandas as pd

"""
Helper class to simplify common read-only BigQuery tasks.
"""


import pandas as pd
import time

from google.cloud import bigquery


class BigQueryHelper(object):
    """
    Helper class to simplify common BigQuery tasks like executing queries,
    showing table schemas, etc without worrying about table or dataset pointers.

    See the BigQuery docs for details of the steps this class lets you skip:
    https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/reference.html
    """

    def __init__(self, active_project, dataset_name, max_wait_seconds=180):
        self.project_name = active_project
        self.dataset_name = dataset_name
        self.max_wait_seconds = max_wait_seconds
        self.client = bigquery.Client()
        self.__dataset_ref = self.client.dataset(self.dataset_name, project=self.project_name)
        self.dataset = None
        self.tables = dict()  # {table name (str): table object}
        self.__table_refs = dict()  # {table name (str): table reference}
        self.total_gb_used_net_cache = 0
        self.BYTES_PER_GB = 2**30

    def __fetch_dataset(self):
        """
        Lazy loading of dataset. For example,
        if the user only calls `self.query_to_pandas` then the
        dataset never has to be fetched.
        """
        if self.dataset is None:
            self.dataset = self.client.get_dataset(self.__dataset_ref)

    def __fetch_table(self, table_name):
        """
        Lazy loading of table
        """
        self.__fetch_dataset()
        if table_name not in self.__table_refs:
            self.__table_refs[table_name] = self.dataset.table(table_name)
        if table_name not in self.tables:
            self.tables[table_name] = self.client.get_table(self.__table_refs[table_name])

    def __handle_record_field(self, row, schema_details, top_level_name=''):
        """
        Unpack a single row, including any nested fields.
        """
        name = row['name']
        if top_level_name != '':
            name = top_level_name + '.' + name
        schema_details.append([{
            'name': name,
            'type': row['type'],
            'mode': row['mode'],
            'fields': pd.np.nan,
            'description': row['description']
                               }])
        # float check is to dodge row['fields'] == np.nan
        if type(row.get('fields', 0.0)) == float:
            return None
        for entry in row['fields']:
            self.__handle_record_field(entry, schema_details, name)

    def __unpack_all_schema_fields(self, schema):
        """
        Unrolls nested schemas. Returns dataframe with one row per field,
        and the field names in the format accepted by the API.
        Results will look similar to the website schema, such as:
            https://bigquery.cloud.google.com/table/bigquery-public-data:github_repos.commits?pli=1

        Args:
            schema: DataFrame derived from api repr of raw table.schema
        Returns:
            Dataframe of the unrolled schema.
        """
        schema_details = []
        schema.apply(lambda row:
            self.__handle_record_field(row, schema_details), axis=1)
        result = pd.concat([pd.DataFrame.from_dict(x) for x in schema_details])
        result.reset_index(drop=True, inplace=True)
        del result['fields']
        return result

    def table_schema(self, table_name):
        """
        Get the schema for a specific table from a dataset.
        Unrolls nested field names into the format that can be copied
        directly into queries. For example, for the `github.commits` table,
        the this will return `committer.name`.

        This is a very different return signature than BigQuery's table.schema.
        """
        self.__fetch_table(table_name)
        raw_schema = self.tables[table_name].schema
        schema = pd.DataFrame.from_dict([x.to_api_repr() for x in raw_schema])
        # the api_repr only has the fields column for tables with nested data
        if 'fields' in schema.columns:
            schema = self.__unpack_all_schema_fields(schema)
        # Set the column order
        schema = schema[['name', 'type', 'mode', 'description']]
        return schema

    def list_tables(self):
        """
        List the names of the tables in a dataset
        """
        self.__fetch_dataset()
        return([x.table_id for x in self.client.list_tables(self.dataset)])

    def estimate_query_size(self, query):
        """
        Estimate gigabytes scanned by query.
        Does not consider if there is a cached query table.
        See https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
        """
        my_job_config = bigquery.job.QueryJobConfig()
        my_job_config.dry_run = True
        my_job = self.client.query(query, job_config=my_job_config)
        return my_job.total_bytes_processed / self.BYTES_PER_GB

    def query_to_pandas(self, query):
        """
        Execute a SQL query & return a pandas dataframe
        """
        my_job = self.client.query(query)
        start_time = time.time()
        while not my_job.done():
            if (time.time() - start_time) > self.max_wait_seconds:
                print("Max wait time elapsed, query cancelled.")
                self.client.cancel_job(my_job.job_id)
                return None
            time.sleep(0.1)
        # Queries that hit errors will return an exception type.
        # Those exceptions don't get raised until we call my_job.to_dataframe()
        # In that case, my_job.total_bytes_billed can be called but is None
        if my_job.total_bytes_billed:
            self.total_gb_used_net_cache += my_job.total_bytes_billed / self.BYTES_PER_GB
        return my_job.to_dataframe()

    def query_to_pandas_safe(self, query, max_gb_scanned=1):
        """
        Execute a query, but only if the query would scan less than `max_gb_scanned` of data.
        """
        query_size = self.estimate_query_size(query)
        if query_size <= max_gb_scanned:
            return self.query_to_pandas(query)
        msg = "Query cancelled; estimated size of {0} exceeds limit of {1} GB"
        print(msg.format(query_size, max_gb_scanned))

    def head(self, table_name, num_rows=5, start_index=None, selected_columns=None):
        """
        Get the first n rows of a table as a DataFrame.
        Does not perform a full table scan; should use a trivial amount of data as long as n is small.
        """
        self.__fetch_table(table_name)
        active_table = self.tables[table_name]
        schema_subset = None
        if selected_columns:
            schema_subset = [col for col in active_table.schema if col.name in selected_columns]
        results = self.client.list_rows(active_table, selected_fields=schema_subset,
            max_results=num_rows, start_index=start_index)
        results = [x for x in results]
        return pd.DataFrame(
            data=[list(x.values()) for x in results], columns=list(results[0].keys()))

# Viewing table schema and data

In [0]:
illu = BigQueryHelper('bigquery-public-data','human_genome_variants')

In [0]:
illu.list_tables()

['1000_genomes_pedigree',
 '1000_genomes_phase_3_optimized_schema_variants_20150220',
 '1000_genomes_phase_3_variants_20150220',
 '1000_genomes_sample_info',
 'platinum_genomes_deepvariant_variants_20180823',
 'simons_genome_diversity_project_sample_attributes',
 'simons_genome_diversity_project_sample_metadata',
 'simons_genome_diversity_project_sample_variants']

In [0]:
#@title Default title text
illu.table_schema('platinum_genomes_deepvariant_variants_20180823')

,name,type,mode,description
0,reference_name,STRING,NULLABLE,Reference name.
1,start_position,INTEGER,NULLABLE,Start position (0-based). Corresponds to the f...
2,end_position,INTEGER,NULLABLE,End position (0-based). Corresponds to the fir...
3,reference_bases,STRING,NULLABLE,Reference bases.
4,alternate_bases,RECORD,REPEATED,One record for each alternate base (if any).
5,alternate_bases.alt,STRING,NULLABLE,Alternate base.
6,names,STRING,REPEATED,Variant names (e.g. RefSNP ID).
7,quality,FLOAT,NULLABLE,Phred-scaled quality score (-10log10 prob(call...
8,filter,STRING,REPEATED,"List of failed filters (if any) or ""PASS"" indi..."
9,call,RECORD,REPEATED,One record for each call.


In [0]:
illu.head('platinum_genomes_deepvariant_variants_20180823',num_rows=10)

,reference_name,start_position,end_position,reference_bases,alternate_bases,names,quality,filter,call,partition_date_please_ignore
0,chr1,2118723,2118724,C,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
1,chr1,7916656,7916657,T,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
2,chr1,2615293,2615294,C,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
3,chr1,7966480,7966481,A,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
4,chr1,101136,101137,A,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
5,chr1,1286760,1286761,A,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
6,chr1,7494953,7494954,A,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
7,chr1,2605319,2605320,G,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
8,chr1,1000332,1000334,T,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23
9,chr1,823783,823784,T,[{'alt': '<*>'}],[],0.0,[],"[{'name': 'NA12877', 'genotype': [0, 0], 'phas...",2018-08-23


In [0]:
print('Size of dataframe: {} Bytes'.format(int(illu.memory_usage(index=True, deep=True).sum())))

# Querying the table

## Count total number of rows

In [0]:
# counting total number of rows
# scan size 0.0 GB, result = 105923159 rows
NUM_ROWS =  """
            SELECT
            COUNT(1) AS num_rows
            FROM 
              `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823`
            """

In [0]:
illu.estimate_query_size(NUM_ROWS)

0.0

In [0]:
df = illu.query_to_pandas_safe(NUM_ROWS)

In [0]:
df

,num_rows
0,105923159


## Counting variant calls in the table
__Summing the length of call arrays__

In [0]:
# counting total number of variant calls 
# scan size 0.0 GB, result=182104652 variant calls
# why 1.7 variat calls per row?
NUM_VARIANTCALLS =  """
                    SELECT
                      SUM(ARRAY_LENGTH(call)) AS num_calls
                    FROM 
                      `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823`
                    """

In [0]:
illu.estimate_query_size(NUM_VARIANTCALLS)

0.0

In [0]:
df = illu.query_to_pandas_safe(NUM_VARIANTCALLS)

In [0]:
df

,num_calls
0,182104652


__Joining each row__

In [0]:
## scan size 0.0. GB
NUM_VARIANTCALLS2 =  """
                    SELECT
                      COUNT(call) AS num_calls
                    FROM 
                      `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                    """

In [0]:
illu.estimate_query_size(NUM_VARIANTCALLS2)

0.0

__Counting name in a call column__

In [0]:
## scan size ~ 1.5264 GB
NUM_VARIANTCALLS3 = """
                    SELECT
                      COUNT(call.name) AS num_calls
                    FROM
                      `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
                    """

In [0]:
illu.estimate_query_size(NUM_VARIANTCALLS3)

1.5263835601508617

## Counting variant and non-variant segments
__non_variant segments__

In [0]:
# counting non-variant segments
# scan size 0.534 GB, returns 143.555.264 non-variant segments
NUM_NONVARIANTS =  """
                SELECT
                  COUNT(1) AS num_nonvariants
                FROM
                  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
                WHERE
                  NOT EXISTS (SELECT 1
                                FROM UNNEST(v.alternate_bases) AS alt
                              WHERE
                                alt.alt NOT IN ('<NON_REF>','<*>'))
                """

In [0]:
illu.estimate_query_size(NUM_NONVARIANTS)

0.5340448450297117

In [0]:
df = illu.query_to_pandas_safe(NUM_NONVARIANTS)

In [0]:
df

,num_variants
0,143555264


__variant segments__

In [0]:
# counting variant segments
# scan size 0.534 GB, returns 38.549.388 variant segments
NUM_VARIANTS =  """
                SELECT
                  COUNT(1) AS num_variants
                FROM
                  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
                WHERE
                  EXISTS (SELECT 1
                            FROM UNNEST(v.alternate_bases) AS alt
                          WHERE
                            alt.alt NOT IN ('<NON_REF>','<*>'))
                """

In [0]:
illu.estimate_query_size(NUM_VARIANTS)

0.5340448450297117

In [0]:
df = illu.query_to_pandas_safe(NUM_VARIANTS)
df

,num_variants
0,38549388


In [0]:
38549388+143555264

182104652

## Counting the variants called by each sample

In [0]:
# counting number of variants called by each sample
# first count number of rows, in which each call set appears
# scan size 1.526 GB, returns call counts of approx. 30 million variant calls per sample
CALLCOUNT_PER_SET =   """
                            SELECT
                              call.name AS call_names,
                              COUNT(call.name) AS call_count_per_call_set
                            FROM
                              `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                            GROUP BY 
                              call.name
                            ORDER BY
                              call.name
                            """

In [0]:
illu.estimate_query_size(CALLCOUNT_PER_SET)

1.5263835601508617

In [0]:
df = illu.query_to_pandas(CALLCOUNT_PER_SET)
df

,call_names,call_count_per_call_set
0,NA12877,31592135
1,NA12878,28012646
2,NA12889,31028550
3,NA12890,30636087
4,NA12891,33487348
5,NA12892,27347886


In [0]:
## filter out non-variant segments to only count the called variant segments per sample (human)
# scan size 2.06 GB, returns around 6 million called variant segments per sample (more realistic)
NUM_VARIANTS_PER_SAMPLE =   """
                            SELECT
                              call.name AS call_name,
                              COUNT(call.name) AS call_count_for_call_set
                            FROM
                              `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                            WHERE
                              EXISTS (SELECT 1
                                        FROM UNNEST(v.alternate_bases) AS alt
                                      WHERE
                                        alt.alt NOT IN ("<NON_REF>", "<*>"))
                            GROUP BY
                              call_name
                            ORDER BY
                              call_name
                            """

In [0]:
illu.estimate_query_size(NUM_VARIANTS_PER_SAMPLE)

2.0604284051805735

In [0]:
df = illu.query_to_pandas(NUM_VARIANTS_PER_SAMPLE)

In [0]:
df

,call_name,call_count_for_call_set
0,NA12877,6284275
1,NA12878,6397315
2,NA12889,6407532
3,NA12890,6448600
4,NA12891,6516669
5,NA12892,6494997


__Filtering true variants by genotype__

In [0]:
# filtering true variants by genotype
# variants are no-calls, if genotype = -1
# true variants for individuals have genotype > 0
# if a call includes only no-call genotypes or reference genotypes (0), they are not true variants
# BASED ON QUERY I ASSUME THAT WE CALL REFERENCE (GENOTYPE=0) VARIANTS AS WELL ???
NUM_TRUE_VARIANTS_PER_SAMPLE = """
                                SELECT 
                                  call.name AS call_name,
                                  COUNT(call.name) AS call_count_per_call_set
                                FROM 
                                  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                                WHERE
                                  EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt>0)
                                  AND NOT EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt<0)
                                GROUP BY 
                                  call_name
                                ORDER BY  
                                  call_name
                                """
# scan size 4.2399 GB, results in around 4.5 million true variants per call/samples

In [0]:
illu.estimate_query_size(NUM_TRUE_VARIANTS_PER_SAMPLE)

4.239954333752394

In [0]:
df = illu.query_to_pandas(NUM_TRUE_VARIANTS_PER_SAMPLE)
df

,call_name,call_count_per_call_set
0,NA12877,4486610
1,NA12878,4502017
2,NA12889,4422706
3,NA12890,4528725
4,NA12891,4424094
5,NA12892,4495753


## Count samples in the table

In [0]:
# count number of samples (calls) in data set
# scan size 1.5264 GB, returns 6
NUM_SAMPLES = """
              SELECT
                COUNT(DISTINCT(call.name)) AS num_callsets
              FROM 
                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
              """

In [0]:
illu.estimate_query_size(NUM_SAMPLES)

1.5263835601508617

In [0]:
df = illu.query_to_pandas(NUM_SAMPLES)
df

,num_callsets
0,6


## Count variants per chromosome

In [0]:
# counting variants per chromosome
# WHY DOES HERE ONLY ONE GT HAVE TO BE >0 FOR THE VARIANT TO BE A TRUE ONE?
NUM_VARIANTS_PER_CHR =  """
                        SELECT 
                          reference_name,
                          COUNT(reference_name) AS num_variant_rows
                        FROM
                          `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                        WHERE
                          EXISTS (SELECT 1 
                                  FROM 
                                    UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                                  WHERE
                                    gt>0)
                        GROUP BY
                          reference_name
                        ORDER BY
                          CASE
                            WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
                              THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
                              ELSE REGEXP_REPLACE(reference_name, '^chr', '')
                          END
                        """
# scan size 3.341 GB, returns number of (true) variants per chr (max. 615000 for chr, min. 15357 for chrY )

In [0]:
illu.estimate_query_size(NUM_VARIANTS_PER_CHR)

3.341447635553777

In [0]:
df = illu.query_to_pandas(NUM_VARIANTS_PER_CHR)
df

,reference_name,num_variant_rows
0,chr1,615000
1,chr2,646401
2,chr3,542315
3,chr4,578600
4,chr5,496202
5,chr6,512152
6,chr7,459506
7,chr8,416376
8,chr9,344985
9,chr10,396773


## Count high-quality variants per sample
__Querying calls with multiple FILTER values__

In [0]:
# counting high-quality calls per sample
# filter column labels variant calls of differing qualities (list of failed filters or PASS if HQ)
NUM_VARIANTS_PER_FILTER =   """
                            SELECT
                              call_filter,
                              COUNT(call_filter) AS number_of_calls
                            FROM
                              `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v,
                              v.call,
                              UNNEST(call.FILTER) AS call_filter
                            GROUP BY
                              call_filter
                            ORDER BY
                              number_of_calls
                            """
# scan size 0.248 GB, returns 11681534 counts for RefCall and 26867854 for PASS

In [0]:
illu.estimate_query_size(NUM_VARIANTS_PER_FILTER)

0.24804932065308094

In [0]:
df = illu.query_to_pandas(NUM_VARIANTS_PER_FILTER)
df

,call_filter,number_of_calls
0,RefCall,11681534
1,PASS,26867854


__FILTERing for high quality variant calls__

In [0]:
# proof that if a variant call passed all qualities filters (filter=PASS), the filter field does not contain any other value
PASS_IS_PASS =  """
                SELECT
                  reference_name,
                  start_position,
                  end_position,
                  reference_bases,
                  call.name AS call_name,
                  (SELECT STRING_AGG(call_filter) FROM UNNEST(call.FILTER) AS call_filter) AS filters,
                  ARRAY_LENGTH(call.FILTER) AS filter_count
                FROM
                  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                WHERE
                  EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter = 'PASS')
                  AND ARRAY_LENGTH(call.FILTER) > 1
                ORDER BY
                  filter_count DESC, reference_name, start_position, end_position, reference_bases, call_name
                LIMIT
                  10
                """
# scan size 4.28 GB, returns None

In [0]:
illu.estimate_query_size(PASS_IS_PASS)

4.281298340298235

In [0]:
df = illu.query_to_pandas(PASS_IS_PASS)
df

,reference_name,start_position,end_position,reference_bases,call_name,filters,filter_count


__Counting all high quality calls for each sample__

In [0]:
# counting all high-quality calls for each sample
NUM_HQ_PER_SAMPLE = """
                    SELECT
                      call.name AS call_name,
                      COUNT(1) AS num_of_calls
                    FROM 
                      `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                    WHERE
                      NOT EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter != 'PASS')
                    GROUP BY
                      call_name
                    ORDER BY
                      call_name
                    """
# scan size 1.774 GB, returns number of hq calls per sample min/max?

In [0]:
illu.estimate_query_size(NUM_HQ_PER_SAMPLE)

1.7744328808039427

In [0]:
df = illu.query_to_pandas(NUM_HQ_PER_SAMPLE)
df

,call_name,num_of_calls
0,NA12877,29795946
1,NA12878,26118774
2,NA12889,29044992
3,NA12890,28717437
4,NA12891,31395995
5,NA12892,25349974


__Counting all high quality true variant calls for each sample__

In [0]:
# counting hq true variant calls per sample
NUM_HQ_TRUE_V_PER_SAMPLE =  """
                            SELECT
                              call.name AS call_name,
                              COUNT(1) AS num_calls
                            FROM 
                              `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
                            WHERE
                              NOT EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter != 'PASS')
                              AND EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt>0)
                            GROUP BY
                              call_name
                            ORDER BY
                              call_name
                            """
# scan size 4.488 GB, returns the same table as NUM_TRUE_VARIANTS_PER_SAMPLE (seems like all true variants are also HQ)

In [0]:
illu.estimate_query_size(NUM_HQ_TRUE_V_PER_SAMPLE)

4.488003654405475

In [0]:
df = illu.query_to_pandas(NUM_HQ_TRUE_V_PER_SAMPLE)
df

,call_name,num_calls
0,NA12877,4486610
1,NA12878,4502017
2,NA12889,4422706
3,NA12890,4528725
4,NA12891,4424094
5,NA12892,4495753


## Best practices
__Condesing queries__

In [0]:
# Best practices - condensing queries
# from here step-wise to NUM_VARIANTS_PER_CHR query
NUM_VARAINTS_PER_CHR_CNDSD1 =  """
                              SELECT
                                reference_name,
                                COUNT(reference_name) AS number_of_variant_rows
                              FROM
                                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                              WHERE
                                EXISTS (SELECT 1
                                          FROM UNNEST(v.call) AS call
                                        WHERE EXISTS (SELECT 1
                                                        FROM UNNEST(call.genotype) AS gt
                                                      WHERE gt > 0))
                              GROUP BY
                                reference_name
                              ORDER BY
                                reference_name
                              """
# scan size  3.3414 GB, returns number of variants per chr but not numerically sorted        

In [0]:
illu.estimate_query_size(NUM_VARAINTS_PER_CHR_CNDSD1)

3.341447635553777

In [0]:
# make query more concise by changing the EXISTS clause into a JOIN of the call column 
# with the call.genotype column. Recall that the comma operator is a shorthand notation used for JOIN
NUM_VARAINTS_PER_CHR_CNDSD2 = """
                              SELECT
                                reference_name,
                                COUNT(reference_name) AS number_of_variant_rows
                              FROM
                                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                              WHERE
                                EXISTS (SELECT 1
                                          FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                                        WHERE gt > 0)
                              GROUP BY
                                reference_name
                              ORDER BY
                                reference_name
                              """
# scan size 3.3414 GB, returns number of variants per chr but not numerically ordered according to chr name

In [0]:
illu.estimate_query_size(NUM_VARAINTS_PER_CHR_CNDSD2)

3.341447635553777

In [0]:
# # to sort numerically, remove 'chr' from chr reference name
# chromosome alias
NUM_VARAINTS_PER_CHR_CNDSD3 = """
                              SELECT
                                REGEXP_REPLACE(reference_name, '^chr', '') AS chromosome,
                                COUNT(reference_name) AS number_of_variant_rows
                              FROM
                                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                              WHERE
                                EXISTS (SELECT 1
                                          FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                                        WHERE gt > 0)
                              GROUP BY
                                chromosome
                              ORDER BY
                                chromosome
                              """
# scan size 3.3414 GB

In [0]:
illu.estimate_query_size(NUM_VARAINTS_PER_CHR_CNDSD3)

3.341447635553777

In [0]:
# to sort numerically, cast str to integer
NUM_VARAINTS_PER_CHR_CNDSD4 = """
                              SELECT
                                CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) AS chromosome,
                                COUNT(reference_name) AS number_of_variant_rows
                              FROM
                                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                              WHERE
                                EXISTS (SELECT 1
                                          FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                                        WHERE gt > 0)
                              GROUP BY
                                chromosome
                              ORDER BY
                                chromosome
                              """

In [0]:
# X, Y, M chromosomes need to bee sorted by string and not numerically!!!
# CASE and prepend 0 to chr 1 trough 9
# chromosome alias
NUM_VARAINTS_PER_CHR_CNDSD5 = """
                              SELECT
                                CASE
                                  WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
                                    THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
                                    ELSE REGEXP_REPLACE(reference_name, '^chr', '')
                                END AS chromosome,
                                COUNT(reference_name) AS number_of_variant_rows
                              FROM
                                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                              WHERE
                                EXISTS (SELECT 1
                                          FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                                        WHERE gt > 0)
                              GROUP BY
                                chromosome
                              ORDER BY
                                chromosome
                              """
# scan size -"- GB

In [0]:
illu.estimate_query_size(NUM_VARAINTS_PER_CHR_CNDSD4)

3.341447635553777

In [0]:
# Note the use of the SAFE_CAST function, which returns NULL for chromosomes X, Y, and M 
# instead of returning an error.
# As a last improvement on the output, display the reference_name column again
#instead of setting it to the chromosome alias. To do so, move the CASE clause to the ORDER BY function
NUM_VARAINTS_PER_CHR_CNDSD5 = """
                              SELECT
                                reference_name,
                                COUNT(reference_name) AS number_of_variant_rows
                              FROM
                                `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                              WHERE
                                EXISTS (SELECT 1
                                          FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                                        WHERE gt > 0)
                              GROUP BY
                                reference_name
                              ORDER BY
                                CASE
                                  WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
                                    THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
                                    ELSE REGEXP_REPLACE(reference_name, '^chr', '')
                                END
                              """
# == NUM_VARAINTS_PER_CHR

__Writing user-defined functions__

In [0]:
# Best practices - Wrtiting user defined functions
# make complex queries (as the one above) less verbose by putting case expression into a function
# making query more concise
USER_FUNCTION = """
                CREATE TEMPORARY FUNCTION SortableChromosome(reference_name STRING)
                  RETURNS STRING AS (
                  -- Remove the leading "chr" (if any) in the reference_name
                  -- If the chromosome is 1 - 9, prepend a "0" since
                  -- "2" sorts after "10", but "02" sorts before "10".
                  CASE
                    WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
                      THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
                      ELSE REGEXP_REPLACE(reference_name, '^chr', '')
                  END
                );

                SELECT
                  reference_name,
                  COUNT(reference_name) AS number_of_variant_rows
                FROM
                  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
                WHERE
                  EXISTS (SELECT 1
                            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
                          WHERE gt > 0)
                GROUP BY
                  reference_name
                ORDER BY SortableChromosome(reference_name)
              """
# scan size -----

In [0]:
illu.estimate_query_size(USER_FUNCTION)

3.341447635553777

Improving query performance and reducing costs by trying different ways of writing a query and choosing the most efficient one. <br>
Deleting or turning off resources on GCP to prevent them from taking up quota in the future.